In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn
import statistics
import math
from scipy.stats import poisson,skellam

world_cup_matches = pd.read_csv("C:\\Users\\ianfr\\Documents\\dev\\inteligencia-computacional\\dataset\\results.csv")
world_cup_matches['year'] = world_cup_matches['date'].str[:4].astype(int)
world_cup_matches['month'] = world_cup_matches['date'].str[5:7].astype(int)
world_cup_matches['day'] = world_cup_matches['date'].str[8:].astype(int)
world_cup_matches = world_cup_matches[['home_team', 'home_score', 'away_score', 'away_team', 'tournament', 'year', 'month', 'day']]
world_cup_matches = world_cup_matches[(world_cup_matches.year) >= 2014]
world_cup_matches = world_cup_matches[(world_cup_matches.year) <= 2018]

wc_teams = ['Russia', 'Brazil', 'Australia', 'Iran', 'Japan', 'Saudi Arabia', 'South Korea', 'Egypt', 'Morocco', 'Nigeria', 'Senegal', 'Tunisia', 'Costa Rica', 'Mexico', 'Panama', 'Argentina', 'Colombia', 'Peru', 'Uruguay', 'Belgium', 'Croatia', 'Denmark', 'England', 'France', 'Germany', 'Iceland', 'Poland', 'Portugal', 'Serbia', 'Spain', 'Sweden', 'Switzerland']
wc_teams.sort()
#world_cup_matches = world_cup_matches[world_cup_matches.home_team.isin(teams) | world_cup_matches.away_team.isin(teams)]

world_cup_matches.head()

,home_team,home_score,away_score,away_team,tournament,year,month,day
35750,Kuwait,1,2,Jordan,WAFF Championship,2014,1,1
35751,Bahrain,0,1,Jordan,WAFF Championship,2014,1,4
35752,Namibia,0,1,Ghana,Friendly,2014,1,4
35753,Nigeria,2,1,Ethiopia,Friendly,2014,1,4
35754,Qatar,3,0,Kuwait,WAFF Championship,2014,1,4


In [7]:
world_cup_matches.describe()

,home_score,away_score,year,month,day
count,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000
mean,1.553141,1.097153,2015.988481,7.146273,14.808085
std,1.541122,1.346297,1.384416,3.169644,8.781769
min,0.000000,0.000000,2014.000000,1.000000,1.000000
25%,0.000000,0.000000,2015.000000,5.000000,7.000000
50%,1.000000,1.000000,2016.000000,7.000000,13.000000
75%,2.000000,2.000000,2017.000000,10.000000,23.000000
max,15.000000,20.000000,2018.000000,12.000000,31.000000


In [8]:
teams2 = world_cup_matches['home_team'].unique().tolist()
teams3 = world_cup_matches['away_team'].unique().tolist()
tournaments = world_cup_matches['tournament'].unique().tolist()
print(tournaments)
game_weight = {'Friendly': 1, 'FIFA World Cup qualification': 2.5, 'UEFA Euro': 2.5, 'Copa América': 2.5,
               'UEFA Nations League': 2.5, 'Oceania Nations Cup': 2.5, 'Confederations Cup': 3, 'FIFA World Cup': 4}

teams = list(set(teams2).union(set(teams3)))
data = {'Team': teams, 'AttackForce': [0 for i in range(len(teams))], 'DefenseForce': [0 for i in range(len(teams))]}
stats = pd.DataFrame(data)

for time in teams:
    team_goals = 0.0
    conceded_goals = 0.0
    team_matches = 0.0
    time_model = world_cup_matches[(world_cup_matches['home_team'] == time) | (world_cup_matches['away_team'] == time)]
    #print(time_model)
    for index, row in time_model.iterrows():
        time_factor = math.exp(-(2018 - row['year'])/3)
        
        if row['tournament'] in game_weight:
            game_weight_factor = game_weight[row['tournament']]
        else:
            game_weight_factor = 2

        #print(game_weight_factor)
        if row['home_team'] == time:
            team_goals += row['home_score'] * time_factor * game_weight_factor
            conceded_goals += row['away_score'] * time_factor * game_weight_factor
            
            if time == 'Brazil':
                print(conceded_goals, row['away_score'], time_factor, game_weight_factor)

        if row['away_team'] == time:
            team_goals += row['away_score'] * time_factor * game_weight_factor
            conceded_goals += row['home_score'] * time_factor * game_weight_factor
            
            if time == 'Brazil':
                print(conceded_goals, row['home_score'], time_factor, game_weight_factor)
            
        team_matches += 1
        
    
    if conceded_goals != 0 and team_goals != 0:
        stats.loc[stats['Team'] == time, 'AttackForce'] = team_goals/(team_matches)
        stats.loc[stats['Team'] == time, 'DefenseForce'] = (team_matches/(conceded_goals))
    else:
        print(time)
        stats = stats[stats['Team'] != time]
        world_cup_matches = world_cup_matches[world_cup_matches['home_team'] != time]
        world_cup_matches = world_cup_matches[world_cup_matches['away_team'] != time]
    
stats2 = stats.set_index('Team')

print(stats2)

world_cup_matches['HomeAttack'] = [0.0 for i in range(len(world_cup_matches))]
world_cup_matches['HomeDefence'] = [0.0 for i in range(len(world_cup_matches))]
world_cup_matches['AwayAttack'] = [0.0 for i in range(len(world_cup_matches))]
world_cup_matches['AwayDefence'] = [0.0 for i in range(len(world_cup_matches))]

for index, row in world_cup_matches.iterrows():
    #print(stats2.loc[row['home_team'], 'AttackForce'])
    world_cup_matches.loc[index,'HomeAttack'] = (stats2.loc[row['home_team'], 'AttackForce']) #- statistics.mean(stats2['AttackForce'])) / statistics.stdev(stats2['AttackForce'])
    world_cup_matches.loc[index,'HomeDefence'] = (stats2.loc[row['home_team'], 'DefenseForce']) #- statistics.mean(stats2['DefenseForce'])) / statistics.stdev(stats2['DefenseForce'])
    world_cup_matches.loc[index,'AwayAttack'] = (stats2.loc[row['away_team'], 'AttackForce']) #- statistics.mean(stats2['AttackForce'])) / statistics.stdev(stats2['AttackForce'])
    world_cup_matches.loc[index,'AwayDefence'] = (stats2.loc[row['away_team'], 'DefenseForce']) #- statistics.mean(stats2['DefenseForce'])) / statistics.stdev(stats2['DefenseForce'])
world_cup_matches.head()

['WAFF Championship', 'Friendly', 'African Nations Championship', 'AFC Asian Cup qualification', 'African Cup of Nations qualification', 'Windward Islands Tournament', 'AFC Challenge Cup', 'Baltic Cup', 'CFU Caribbean Cup qualification', 'CONIFA World Football Cup', 'FIFA World Cup', 'EAFF Championship', 'UNCAF Cup', 'UEFA Euro qualification', 'AFF Championship', 'CFU Caribbean Cup', 'Gulf Cup', 'AFC Asian Cup', 'African Cup of Nations', 'ABCS Tournament', 'FIFA World Cup qualification', 'Gold Cup qualification', 'COSAFA Cup', 'Copa América', 'CONIFA European Football Cup', 'Island Games', 'Gold Cup', 'Oceania Nations Cup qualification', 'CECAFA Cup', 'SAFF Cup', 'Copa América qualification', 'Oceania Nations Cup', 'Kirin Cup', "King's Cup", 'UEFA Euro', 'World Unity Cup', 'Confederations Cup', 'Intercontinental Cup', 'AFF Championship qualification', 'UEFA Nations League']
0.0 0 0.26359713811572677 1
0.0 0 0.26359713811572677 1
0.0 0 0.26359713811572677 1
1.054388552462907 1 0.2635971

,home_team,home_score,away_score,away_team,tournament,year,month,day,HomeAttack,HomeDefence,AwayAttack,AwayDefence
35750,Kuwait,1,2,Jordan,WAFF Championship,2014,1,1,0.729723,1.134056,1.158306,1.484830
35751,Bahrain,0,1,Jordan,WAFF Championship,2014,1,4,1.110792,1.636862,1.158306,1.484830
35752,Namibia,0,1,Ghana,Friendly,2014,1,4,0.932738,0.979487,1.264597,1.304191
35753,Nigeria,2,1,Ethiopia,Friendly,2014,1,4,1.386389,0.952388,1.093469,0.753695
35754,Qatar,3,0,Kuwait,WAFF Championship,2014,1,4,1.262532,1.113806,0.729723,1.134056


In [9]:
stats2.describe()

,AttackForce,DefenseForce
count,271.000000,271.000000
mean,1.389916,0.802017
std,0.781365,0.382830
min,0.065899,0.125000
25%,0.930081,0.556685
50%,1.241175,0.776652
75%,1.685048,0.982048
max,5.666667,3.161390


In [10]:
for team in wc_teams:
    print(team, "Attack: " + str(stats2.loc[team, 'AttackForce']), "Defense: " + str(stats2.loc[team, 'DefenseForce']))

Argentina Attack: 1.9298450387083677 Defense: 0.7954194278887741
Australia Attack: 2.047987816151518 Defense: 0.7014676602243811
Belgium Attack: 3.3850050794552353 Defense: 0.8501880587976319
Brazil Attack: 2.33336791079176 Defense: 1.3127929402833414
Colombia Attack: 1.7789430337717878 Defense: 0.963503087187236
Costa Rica Attack: 1.3420216506957736 Defense: 0.7068992668111079
Croatia Attack: 2.3001469163085226 Defense: 0.6334412539332583
Denmark Attack: 1.6852303305417207 Defense: 1.2440909842111685
Egypt Attack: 1.655527994173274 Defense: 0.871392919415705
England Attack: 2.1996049956021198 Defense: 0.8877506667347411
France Attack: 2.3283476614526313 Defense: 0.950782567773494
Germany Attack: 2.516289248558565 Defense: 0.863504517719254
Iceland Attack: 1.4287590089588287 Defense: 0.5826417701300415
Iran Attack: 1.5204796245295555 Defense: 1.6955634960808668
Japan Attack: 2.1127809763408725 Defense: 0.8031970207307069
Mexico Attack: 1.5818325116101066 Defense: 0.7424954851519918
Mor

In [11]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

# essa tabela duplica os jogos, para ter apenas uma variavel alvo: gols
# ela transforma as colunas em time, oponente, gols, fora o resto das variaveis
# ou seja, os gols sao sempre referentes ao "time", contra o "oponente"
# nao existe mais dentro e fora de casa, ja que a tabela foi duplicada
goal_model_data = pd.concat([world_cup_matches[['home_team','away_team', 'HomeAttack', 'HomeDefence', 'AwayAttack', 'AwayDefence', 'home_score']].rename(
            columns={'home_team':'team', 'away_team':'opponent', 'HomeAttack':'att', 'HomeDefence': 'defe', 'AwayAttack': 'away_att', 'AwayDefence': 'away_def', 'home_score':'goals'}),
           world_cup_matches[['away_team','home_team', 'AwayAttack', 'AwayDefence','HomeAttack', 'HomeDefence', 'away_score']].rename(
            columns={'away_team':'team', 'home_team':'opponent', 'AwayAttack':'att', 'AwayDefence': 'defe', 'HomeAttack': 'away_att', 'HomeDefence': 'away_def','away_score':'goals'})])

#print(goal_model_data)
#goal_model_data = goal_model_data.drop(columns=['team', 'opponent'])
# definicao dos parametros, tendo como variavel alvo os gols e o resto como entrada
poisson_model = smf.glm(formula="goals ~ att + defe + away_att + away_def", data=goal_model_data, 
                        family=sm.families.Poisson()).fit()
#poisson_model.predict()
poisson_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  goals   No. Observations:                 9186
Model:                            GLM   Df Residuals:                     9181
Model Family:                 Poisson   Df Model:                            4
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -13257.
Date:                Fri, 11 Oct 2019   Deviance:                       11454.
Time:                        00:28:31   Pearson chi2:                 1.05e+04
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.5722      0.043     13.170      0.000       0.487       0.657
att            0.4345      0.013     33.647      0.000       0.409       0.460
defe           0.4305      0.032     13.647      0.000       0.369       0.492
away_att      -0.3393      0.017    -20.382      0.000      -0.372      -0.307
away_def      -1.0478      0.034    -31.276      0.000      -1.113      -0.982
==============================================================================
"""

In [13]:
def get_team_stats(team, return_df=False):
    #print(stats2)
    attack = stats2.loc[team, 'AttackForce']
    defence = stats2.loc[team, 'DefenseForce']
    
    # teste para influencia da variavel year
    #year = world_cup_matches.loc[team, 'year']
    #print(attack, defence)
    return [attack, defence]

def simulate_goals(team_pred, game, a_team='A Team', b_team='B Team', verbose=False):
    #a_goals = np.argmax(team_pred[0])
    #b_goals = np.argmax(team_pred[1])
    
    #print(a_goals)
    #print(b_goals)
    a_goals = 0
    b_goals = 0
    outcome = 0

    random_event = np.random.random()
    #print(random_event)
    counter = 0


    for index, row in pd.DataFrame(data=game).iterrows():
        for x in range(len(row)):
            counter += row[x]
            
            a_goals = index
            b_goals = x
            
            if counter > random_event:
                break
        if counter > random_event:
            break
    
    if verbose == True:
        #print('Random float: {}'.format(random_event))
        print('{} goals: {}'.format(a_team, str(a_goals)))
        print('{} goals: {}\n'.format(b_team, str(b_goals)))
    
    return a_goals, b_goals

def show_game_summary(game, team_pred, a_team='A Team', b_team='B Team', verbose=False):
    a_team_odds = np.sum(np.tril(game, -1))
    b_team_odds = np.sum(np.triu(game, 1))
    draw_odds = np.sum(np.diag(game))
    
    if verbose == True:
        print('{} chance of Win: {}%'.format(a_team, round(a_team_odds*100, 2)))
        print('{} chance of Win: {}%'.format(b_team, round(b_team_odds*100, 2)))
        print('Odds of a draw: {}%'.format(round(draw_odds*100, 2)))
    
    a_goals, b_goals = simulate_goals(team_pred, game, a_team, b_team, verbose=verbose)
    #print(a_goals, b_goals)
    return a_team_odds, b_team_odds, draw_odds, a_goals, b_goals

# predicao dos gols dos times em uma partida
def team_preds(foot_model, a_team, b_team, max_goals=5, verbose=False):
    a_team_stats = get_team_stats(a_team)
    b_team_stats = get_team_stats(b_team)

    data1 = {'att': a_team_stats[0],
              'defe': a_team_stats[1],
              'away_att': b_team_stats[0], 
              'away_def': b_team_stats[1]
              }
    print(data1)
    
    data2 = {'att': b_team_stats[0],
              'defe': b_team_stats[1],
              'away_att': a_team_stats[0], 
              'away_def': a_team_stats[1]
            }
    print(data2)
        
    a_goals_avg = foot_model.predict(pd.DataFrame(data=data1, index=[1])).values[0]
    b_goals_avg = foot_model.predict(pd.DataFrame(data=data2, index=[1])).values[0]
    #rint(a_goals_avg)
    #rint(b_goals_avg)
    
    lambda_a = a_team_stats[0] * b_team_stats[1]
    lambda_b = b_team_stats[0] * a_team_stats[1]
    
    team_pred = [[poisson.pmf(i, team_avg) for i in range(0, max_goals+1)] for team_avg in [a_goals_avg, b_goals_avg]]
    #print(max(team_pred[0]))
    #print(team_pred[1])
    #game = (np.outer(np.array(team_pred[0]), np.array(team_pred[1])))
    #print(game)
    #show_game_summary(game, a_team, b_team, verbose=verbose)
    return team_pred

# simulacao de uma partida, retornando a matriz que nos interessa
def simulate_match(team_pred, foot_model, a_team, b_team, max_goals=5, verbose=False):
    game = (np.outer(np.array(team_pred[0]), np.array(team_pred[1])))
    show_game_summary(game, team_pred, a_team, b_team, verbose=verbose)
    return game

In [14]:
team_predz = team_preds(poisson_model, 'Iran', 'Brazil')
game = simulate_match(team_predz, poisson_model, 'Iran', 'Brazil')
#pd.DataFrame(game)
show_game_summary(game, team_predz, 'Iran', 'Brazil', verbose=True)

{'att': 1.5204796245295555, 'defe': 1.6955634960808668, 'away_att': 2.33336791079176, 'away_def': 1.3127929402833414}
{'att': 2.33336791079176, 'defe': 1.3127929402833414, 'away_att': 1.5204796245295555, 'away_def': 1.6955634960808668}
Iran chance of Win: 31.37%
Brazil chance of Win: 34.34%
Odds of a draw: 34.24%
Iran goals: 0
Brazil goals: 0



(0.3136730185208789, 0.34343907396300805, 0.34240018906251624, 0, 0)

In [ ]:
pd.DataFrame(game)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import math

def get_numbers_from_array(array):
    nums = []
    for n in array:
        try:
            nums.append(n[0])
        except:
            nums.append(n)
    return nums

# funcao para calcular as metricas do model
# tenho utilizado como base a metrica r2, que e um valor entre 0 e 1, sendo 1 o modelo perfeito
# o maximo que consegui chegar foi 0.21 e alguma coisa, depois de variar os parametros
# o que ainda e muito baixo, mas nao consegui melhorar muito mais infelizmente
# talvez se nossa predicao fosse de vitoria/derrota poderiamos ter um resultado melhor
def model_metrics(dataframe, poisson_model):
    yreal = []
    ypred = []
    team_pred = []
    i = 0
    print(dataframe)
    yreal = list(dataframe['goals'])
    Y = pd.DataFrame(dataframe['goals'])
    X = dataframe.drop(columns=['goals', 'team', 'opponent'])
    res = sm.Poisson(Y.astype(float), X).fit()
    ypred = poisson_model.predict()
    ypred = get_numbers_from_array(ypred)
    print(ypred)
    #for index, row in dataframe.iterrows():
    #    team_pred = team_preds(poisson_model, row['team'], row['opponent'])
        #print(team_pred)
    #    yreal.append(row['goals'])
    #    ypred.append(team_pred[0].index(max(team_pred[0])))
    #print(yreal)
    #print(ypred)
    rms = math.sqrt(mean_squared_error(yreal, ypred))
    r2 = r2_score(yreal, ypred)
    return rms, r2

metrics = model_metrics(goal_model_data, poisson_model)
print(metrics)

In [ ]:
def normalize_test_model(y):
    if y > (14/3):
        return 5
    elif y > (11/3):
        return 4
    elif y > (8/3):
        return 3
    elif y > (4/3):
        return 2
    elif y < (2/3):
        return 0
    else:
        return 1
    
def compare_accuracy(df):
    count1 = 0

    for index, row in df.iterrows():
        if row['predicted'] == row['goals']:
            count1 += 1
        else:
            continue

    print('Poisson Model: {}% accurate'.format((count1 / len(test_model_df))*100))

#print(goal_model_data)
X = goal_model_data.drop(columns=['goals', 'team', 'opponent'])
Y = pd.DataFrame(goal_model_data['goals'])
#print(X)
#print(np.asarray(X))
res = sm.Poisson(Y.astype(float), X).fit()
poisson_y_pred = poisson_model.predict()
#print(poisson_y_pred)
poisson_y_pred = get_numbers_from_array(poisson_y_pred)
print(poisson_y_pred)
ml_data = [poisson_y_pred]

test_model_df = pd.DataFrame(ml_data, 
                             index=['predicted']).transpose()

test_model_df['goals'] = list(goal_model_data['goals'])
test_model_df['predicted'] = test_model_df['predicted'].apply(normalize_test_model)
print(test_model_df)
compare_accuracy(test_model_df)
#goal_model_data.tail()

In [ ]:
team_predz = team_preds(poisson_model, 'Spain', 'Brazil')
game = simulate_match(team_predz, poisson_model, 'Spain', 'Brazil')
#pd.DataFrame(game)
show_game_summary(game, team_predz, 'Spain', 'Brazil', verbose=True)

In [17]:
def predicao(a_team, b_team, max_goals=5):
    a_team_stats = get_team_stats(a_team)
    b_team_stats = get_team_stats(b_team)
    
    a_att = a_team_stats[0]
    a_def = a_team_stats[1]
    b_att = b_team_stats[0]
    b_def = b_team_stats[1]
    
    a_avg = a_att / b_def
    b_avg = b_att / a_def
    
    print(a_avg)
    print(b_avg)
    
    team_pred = [[poisson_distribution(avg, k) for k in range(0, max_goals+1)] for avg in [a_avg, b_avg]]
    
    return team_pred

def poisson_distribution(avg, k):
    
    result = (math.exp(-avg)*(avg**k)) / math.factorial(k)
    
    return result
    
predicao('Brazil', 'Japan')

2.9051003061104277
1.6093786853278391


[[0.05474329923866765,
  0.15903477537574814,
  0.2310059873131445,
  0.2236985214855859,
  0.1624666608110564,
  0.09439638921098781],
 [0.2000118457720442,
  0.321894801398607,
  0.259025316144378,
  0.138956607587689,
  0.05590845060927283,
  0.01799557374805359]]